In [8]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegressionCV
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
# from sklearn.externals import joblib
import joblib

import warnings
warnings.filterwarnings('ignore')

In [9]:
#显示所有列
pd.set_option('display.max_columns', None)
#显示所有行
pd.set_option('display.max_rows', None)

### 设置不使用科学计数法  #为了直观的显示数字，不采用科学计数法
np.set_printoptions(precision=3, suppress=True)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [10]:
### 一般我们机器学习的流程

# 一、加载数据
#processs_datas/train_process_datas_x.csv
datas_x = pd.read_csv('./processs_datas/train_process_datas_x.csv')
print(datas_x.head())
# print(datas_x.info(verbose=True,null_counts=True))
# processs_datas/train_datas_Y.csv
datas_y = pd.read_csv('./processs_datas/train_datas_Y.csv')
# print(datas_y.head())
# print(datas_y.info())
print(datas_y['happiness'].value_counts())

### 标签里面存在异常值 -8
# ## 一般情况下对于标签存在异常的数据需要删除，注意对应的X也要删除
# ## 先合并X,Y,再删除
# datas = pd.concat([datas_x,datas_y],axis=1)
# # print(datas.head())
# # print(datas.info(verbose=True,null_counts=True))
# datas.replace(-8,np.nan,inplace=True)
# datas.dropna(axis=0,how='any',inplace=True)
# # print(datas.shape)
# # print(datas['happiness'].value_counts(dropna=True))

## 这里我们可以考虑将-8替换成3（特殊情况特殊对待）
datas_y.replace(-8,3,inplace=True)
# print(datas_y.shape)
print(datas_y['happiness'].value_counts())

   id  survey_type  province  city  county  gender  nationality  religion  \
0   1            1        12    32      59       1         1.00      1.00   
1   2            2        18    52      85       1         1.00      1.00   
2   3            2        29    83     126       2         1.00      0.00   
3   4            2        10    28      51       2         1.00      1.00   
4   5            1         7    18      36       2         1.00      1.00   

   religion_freq   edu  edu_status   income  political  floor_area  \
0           1.00 11.00        4.00 20000.00       1.00       45.00   
1           1.00 12.00        4.00 20000.00       1.00      110.00   
2           3.00  4.00        4.00  2000.00       1.00      120.00   
3           1.00  3.00        4.00  6420.00       1.00       78.00   
4           1.00 12.00        1.00 32805.33       2.00       70.00   

   property_0  property_1  property_2  property_3  property_4  property_5  \
0           0           1           0  

In [11]:
# 二、数据清洗与预处理  （这里在dataprocess里面完成了）
# 删除ID字段,无效字段
datas_x.drop(['id'],axis=1,inplace=True)

# 三、获取数据的特征属性X和目标属性Y

# 四、数据分割  
x_train,x_test,y_train,y_test = train_test_split(datas_x,datas_y,test_size=0.2,random_state=11)
# print(x_train.shape)
# print(x_test.shape)

In [12]:
# 五、特征工程  标准化，归一化，降维，特征选择， onehot（一般放在数据处理的时候就去做掉）
# # 做一个标准化
ss = StandardScaler()
### fit_transform   transform
x_train = ss.fit_transform(x_train)
x_test = ss.transform(x_test)
# 六、模型构建
model = LogisticRegressionCV(fit_intercept=True, Cs=np.logspace(-5, 1, 100), 
                          multi_class='multinomial', penalty='l2', solver='lbfgs')
# 七、模型训练
model.fit(x_train,y_train)

LogisticRegressionCV(Cs=array([ 0.00001,  0.00001, ...,  8.69749, 10.     ]),
           class_weight=None, cv=None, dual=False, fit_intercept=True,
           intercept_scaling=1.0, max_iter=100, multi_class='multinomial',
           n_jobs=1, penalty='l2', random_state=None, refit=True,
           scoring=None, solver='lbfgs', tol=0.0001, verbose=0)

In [13]:
# 八、模型评估
print('训练集准确率:',model.score(x_train,y_train))
print('测试集准确率:',model.score(x_test,y_test))
y_train_hat = model.predict(x_train)
y_test_hat = model.predict(x_test)
print('训练集MSE：',mean_squared_error(y_train,y_train_hat))
print('测试集MSE：',mean_squared_error(y_test,y_test_hat))

训练集准确率: 0.6265625
测试集准确率: 0.60125
训练集MSE： 0.57953125
测试集MSE： 0.608125


In [14]:
# 九、模型持久化
## 我们模型持久化的时候，特征工程也要持久化    如果是管道流，保存管道就可以了
## 实际上对于最终我们保存的模型来说，我们可以把所有数据进行训练，再保存模型
datas_x = ss.fit_transform(datas_x)

### 找到最优化的那组超参数的构建的模型对象，用这个模型训练所有的数据
model = LogisticRegressionCV(fit_intercept=True, Cs=np.logspace(-5, 1, 100), 
                          multi_class='multinomial', penalty='l2', solver='lbfgs')
model.fit(datas_x,datas_y)
print(model.score(datas_x,datas_y))
datas_y_hat = model.predict(datas_x)
print(mean_squared_error(datas_y,datas_y_hat))

save_dir_path = './model'
import os
if os.path.exists(save_dir_path):
    pass
else:
    os.makedirs(save_dir_path)
    
joblib.dump(ss,save_dir_path+'/ss.pkl')
joblib.dump(model,save_dir_path+'/logistics.pkl')

0.622125
0.582375


['./model/logistics.pkl']